Librerías necesarias para el proyecto:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPooling2D,Dropout, Input, Concatenate, Lambda, GlobalAveragePooling2D
from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.models import Model

from keras.callbacks import EarlyStopping

import sklearn
from   sklearn.model_selection import train_test_split


In [ ]:
np.random.seed(0)

Acceder al drive con el dataset:

In [ ]:
#Para acceder a nuestros ficheros de Google Drive
from google.colab import drive
drive.mount('/content/drive')
# La carpeta datos (que contiene X_train.npy, y_train.npy, X_test.npy y y_test.npy)
# debe estar en vuestro Drive, dentro de la carpeta 'Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Vamos a mapear cada clase para etiquetar las clases con valores numéricos además de la etiqeuea original que se trataba del nombre en formato string.

In [ ]:
map_labels = {
    'Apple Braeburn':0,
  'Apple Granny Smith':1,
  'Apricot':2,
  'Avocado':3,
  'Banana':4,
  'Blueberry':5,
  'Cactus fruit':6,
  'Cantaloupe':7,
  'Cherry':8,
  'Clementine':9,
  'Corn':10,
  'Cucumber Ripe':11,
  'Grape Blue':12,
  'Kiwi':13,
  'Lemon':14,
  'Limes':15,
  'Mango':16,
  'Onion White':17,
  'Orange':18,
  'Papaya':19,
  'Passion Fruit':20,
  'Peach':21,
  'Pear':22,
  'Pepper Green':23,
  'Pepper Red':24,
  'Pineapple':25,
  'Plum':26,
  'Pomegranate':27,
  'Potato Red':28,
  'Raspberry':29,
  'Strawberry':30,
  'Tomato':31,
  'Watermelon':32
}

Para poder cargar los datos hace falta que se añadan el conjunto de datos al drive personal en la carpeta `drive/MyDrive/Colab Notebooks/datos/`.

In [ ]:
#Primera parte de los datos (parte de entrenamiento)
file_path = 'drive/MyDrive/Colab Notebooks/datos/total_data.npz'

loaded_data = np.load(file_path)
trainX = loaded_data['trainX'] # cargar el array con los valores de cada imagen de train
trainY_names = loaded_data['trainY'] # cargar el array con los valores de las etiquetas de train
testX = loaded_data['testX'] # cargar el array con los valores de cada imagen de test
testY_names = loaded_data['testY'] # cargar el array con los valores de las etiquetas de test


trainY = [map_labels[label] for label in trainY_names]
testY = [map_labels[label] for label in testY_names]


# Trasnformar en arrays de numpy para mayor eficiencia
trainX = np.array(trainX)
trainY = np.array(trainY)
testX = np.array(testX)
testY = np.array(testY)

# Mostrar dimension del conjunto de muestras total
print("Forma de vector trainX de muestras:", trainX.shape)
print("Forma de vector trainY de etiquetas:", trainY.shape)
print("Forma de vector testX de muestras:", testX.shape)
print("Forma de vector testY de etiquetas:", testY.shape)

Forma de vector trainX de muestras: (10112, 100, 100, 3)
Forma de vector trainY de etiquetas: (10112,)
Forma de vector testX de muestras: (6742, 100, 100, 3)
Forma de vector testY de etiquetas: (6742,)


Cada caso representa una imagen de una fruta y posee 100x100 píxeles a color, y por lo tanto los datos de cada caso consisten en una matriz con tres dimensiones de la forma (100,100,3) ya que cada píxel se compone de 3 valores enteros entre 0 y 255 que representan los tres colores Rojo Verde y Azul (RGB).

Aplicamos exactamente el mismo preprocesado de los datos para mantener la homogeneidad de los métodos utlizados y de los resultados

# Preprocesado de los Datos

Vamos a modificar el vector de etiquetas ya que inicialmente cada etiqueta se ha representado como un número
$y_i \in \{0, 1, ..., 33\}$, a partir del mapeado de los nombres de las clases, y por tanto vamos a transformar estas etiquetas al espacio de salida $\mathcal Y$ donde cada etiqueta $y_i \in \mathcal Y$ identifica una clase de nuestro problema $C_{i}$ y es representado por un vecotor de 33 valores binarios donde todos los valores del vector toman el valor 0 excepto el valor en la posición $i$ que toma el valor 1 representando que la etiqueta es la clase $ i \in \{0,1,2,3...33\}$.

Para ello aplicamos One-Hot encoding a las etiquetas empleando la clase OneHotEncoder utilizando los siguiente parámetros para crear el objeto codificado:
*   categories: se especifica la lista categ que contiene las categorías de cada variable.
*   sparse_output=False: se establece en False para obtener una matriz densa en lugar de una matriz dispersa, para obtener una matriz con las columnas de cada nueva variable unificadas para añadirlas directamente al conjunto de datos.
*   drop=None: se establece en None para mantener todas las categorías en cada variable.
Después se utiliza el método fit_transform del objeto codificador para realizar la codificación One-Hot de las variables categóricas elegidas.


La técnica One-Hot encoding se utiliza para evitar que los algoritmos de aprendizaje automático aprendan relaciones entre el orden númerico que existe entre los números consecutivos o rangos que se utilizan para los atributos de los datos. Por eso se utiliza un vector de valores binarios para evitar que las clases puedan inferir relaciones erroneas y confundan al algoritmo a la hora de entrenar con los datos.

In [ ]:
# Etiquetas numéricas
categorias =  [np.arange(len(map_labels))]

# Reshape del vector de etiquetas a una matriz columna para aplicar el codificador al vector
trainY_reshaped = trainY.reshape(-1, 1)
testY_reshaped = testY.reshape(-1, 1)

#Funcion para codificar
encoder = OneHotEncoder(
     categories=categorias,  # Categorías de cada variable
     sparse_output=False,  # crea una matriz sparse cuando se pone TRUE
     drop  = None  #  No quitar categorías en cada variable
     )

trainY_encoded = encoder.fit_transform(trainY_reshaped) # Aplicar One Hot Encoding al conjunto entrenamiento
testY_encoded = encoder.fit_transform(testY_reshaped) # Aplicar One Hot Encoding al conjunto entrenamiento
print("Forma de vector Y del conjunto total de etiquetas:",trainY_encoded.shape)
print("Forma de vector Y del conjunto total de etiquetas:",testY_encoded.shape)

Forma de vector Y del conjunto total de etiquetas: (10112, 33)
Forma de vector Y del conjunto total de etiquetas: (6742, 33)


# Elección de Hiperparametros

##Proceso de selección de modelos (_model selection_)

Un aspecto crucial a tener en cuenta y que hemos mencionado antes, es que el conjunto de datos de test $\mathcal D_{test}$ se __reserva__ para la evaluación del modelo __final__, esto quiere decir que el conjunto de test en ningún momento puede influir en el aprendizaje de nuestro modelo, pues _contaminaría_ los datos e introducirían un sesgo en el modelo; por tanto es importante que no utilicemos estos datos hasta el final del entrenamiento cuando queramos evaluar su rendimiento final, ya que el error en este conjunto de test $E_{test}$ resulta en un buen estimador del error fuera de la muestra $E_{out}$ siempre y cuando los datos se hayan obtenido de una muestra identica e independientemente distribuida para poder aplicar la Inecuación de Hoeffding y garantizar una cota del error de generalización que dependa de $E_{test}$, esta cota es mucho menos suelta que la cota de generalización que depende de $E_{in}$ y por tanto es un buen estimador para validar nuestro modelo siempre que se considere solo la hipótesis final $g$ al final del entrenamiento.

Sin embargo, en el ajuste del modelo nos encontraremos en situaciones donde el modelo cambia drásticamente cuando usamos parámetros en los que influye el entrenamiento; estos parámetros son conocidos como _hiperparámetros_ e influyen de una manera significativa el aprendizaje, entre ellos es común encontrar: el _learning rate_ o _tasa de aprendizaje_ o el tamaño de batch _batch size. Es por esto que debemos de tener alguna forma de evaluar los modelos correspondientes a la elección de los diferentes valores de estos hiperparámetros y poder optar por el mejor de ellos; a este concepto se le denomina elección del modelo o _model selection_. Se trata de poder evaluar _en entrenamiento_ el rendimiento de una serie de modelos cuando se enfrentan con datos __nuevos__, por esto es que usar el propio conjunto de training no es un buen indicador del rendimiento ante datos no vistos por que el modelo esta justamente sesgado por este conjunto, se dice que los datos están _contaminados_. Esto introduce la necesidad de un nuevo conjunto de datos: el conjunto de _validación_.

El conjunto de _validación_ es un conjunto de datos que se extrae del conjunto de datos de entrenamiento, $\mathcal D_{train}$, y que denominaremos conjunto de validación, $\mathcal D_{val}$, con tamaño $K$ de ejemplares. Este conjunto de validación es el que se usará para medir el rendimiento de varios modelos en el proceso de _model selection_ mientras que el conjunto de datos restantes se usará para el ajuste. Es importante notar que para definir este conjunto de validación, no es trivial el tamaño $K$ que elegiremos, de hecho la elección de un buen $K$ debe satisfacer por un lado que hayan suficientes ejemplares de entrenamiento $N-K$, y que a la vez satisfaga un $K$ suficiente para que $\mathcal D_{val}$ estime de la mejor manera ek rendimiento fuera de la muestra. Además existe el problema que nos encontramos cuando queriamos separar training y test, que el ajuste depende de la partición concreta del conjunto de datos y puede dar resultados sesgados. Estos problemas se mejora con la técnica de validación cruzada en $S$ pliegues o _S-fold cross validation_.

La técnica de validación cruzada en $S$ pliegues o S-fold CV consiste en la partición de los datos de entrenamiento $\mathcal D_{train}$ en $S$ grupos (en nuestro caso nos limitaremos a que los grupos sean del mismo tamaño) y que los $S-1$ groupos se usen para ajustar y el restante se use para validar. Este proceso luego es repetido en todas las $S$ posibles formas de realizar el _hold out_ de $\mathcal D_{train}$. Finalmente, se obtiene el promedio de las mediciones de rendimiento de cada _fold_ y esa será la medición final que mide el rendimiento del modelo. En la siguiente figura(2) podemos observar un esquema visual de la partición tanto de training y test, y la obtención de $S$ conjuntos de validación.

![Esquema de la partición en conjuntos de training, validacion y test](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

Un aspecto a tener en cuenta en esta técnica de _model selection_ es el parámetro $S$ que define tanto $K$ como el número de _folds_ o ajustes y validación a realizar del modelo. Un caso extremo es cuando $S = N$, que es lo que se conoce como _leave-one-out_, ya que ajustamos en cada _fold_ dejando un ejemplo fuera para validación; esta configuración es especialmente útil cuando los datos disponibles son escasos ya que con _leave-one-out_ se consigue el mejor ajuste posible por tener el máximo $N-K$ para entrenar, sin embargo, resulta muy costoso el proceso puesto que si $N$ es muy grande, entonces el número de _folds_ a realizar será muy grande ($N$ _folds_). No hay regla universal para elegir $S$, y en la práctica se suelen usar $S=\{5,10\}$ que son considerados buenas opciones gracias a resultados empíricos(1).

> (1) Page 184, An Introduction to Statistical Learning, 2013.
>
> (2) https://scikit-learn.org/stable/_images/grid_search_cross_validation.png

Debido al alto coste computacional, para elegir los hiperparámetros no vamos a utilizar la técnica de cross validation ya que el tiempo necesario para poder entrenar todos los modelos con los distintos hiperparámetros sería excesivo. En cambio vamos a utilizar un único conjunto de validación para comparar el rendimiento de los modelos, que es equivalente a utilizar la técnica de cross validation pero con un solo pligue o fold.

##Selección de mejores hiperparámetros - _Grid Search_

Vamos a utilizar las métricas que hemos explicado al presentar el modelo.

In [ ]:
class CustomSensitivity(tf.keras.metrics.Metric):
    def __init__(self, name='sensitivity', **kwargs):
        super(CustomSensitivity, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight('tp', initializer='zeros')
        self.false_negatives = self.add_weight('fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, dtype=tf.bool)
        y_pred = tf.cast(y_pred > 0.5, dtype=tf.bool)  # Assuming binary classification with a threshold of 0.5

        true_positives = tf.math.logical_and(y_true, y_pred)
        false_negatives = tf.math.logical_and(y_true, tf.math.logical_not(y_pred))

        self.true_positives.assign_add(tf.reduce_sum(tf.cast(true_positives, tf.float32)))
        self.false_negatives.assign_add(tf.reduce_sum(tf.cast(false_negatives, tf.float32)))

    def result(self):
        return self.true_positives / (self.true_positives + self.false_negatives)


In [ ]:
class CustomScoreMetric(tf.keras.metrics.Metric):
    def __init__(self, **kwargs):
        super(CustomScoreMetric, self).__init__(name='custom_score', **kwargs)
        self.cat_accuracy = tf.keras.metrics.CategoricalAccuracy()
        self.f1_score = tf.keras.metrics.F1Score()
        self.custom_sensitivity = CustomSensitivity()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.cat_accuracy.update_state(y_true, y_pred)
        self.f1_score.update_state(y_true, y_pred)
        self.custom_sensitivity.update_state(y_true, y_pred)

    def result(self):
        acc = self.cat_accuracy.result()
        f1 = self.f1_score.result()
        sens = self.custom_sensitivity.result()

        return 0.7 * acc + 0.15 * sens + 0.15 * f1

    def reset_state(self):
        self.cat_accuracy.reset_state()
        self.f1_score.reset_state()
        self.custom_sensitivity.reset_state()


Como hemos definido la técnica que usaremos para entrenar y evaluar un modelo, pasamos a definir el método que usaremos para elegir la mejor configuración de hiperparámetros para un modelo concreto.

Optaremos en nuestro caso por una técnica ampliamente usada para este tipo de selección y es la técnica de _grid search_. _Grid search_ es una de las muchas técnicas para optimizar hiperparámetros de un modelo, lo que también se denomina _hyperparameter tuning_.

Esta técnica en su versión más simple consiste en entrenar y evaluar un modelo (en nuestro caso usando _Cross Validation_) con una serie de hiperparámetros  tal que cada hiperparámetro tenga un conjunto de valores previamente seleccionados; luego, mediante una búsqueda exhaustiva, se obtienen las métricas de cada una de las posibles combinaciones de hiperparámetros obteniendo finalmente un _ranking_ donde elegiremos la configuración con mejores métricas.

Aunque Grid Search permite obtener la mejor configuración de hiperparámetros, nosotros usaremos _grid search_ de manera individual para cada hiperparámetro, es decir, un proceso de _grid search_ uni-dimensional (_grid_ de solo un hiperparámetro) usando un modelo de base (por ejemplo, usando los hiperparámetros por defecto). De esta manera podemos seleccionar el mejor conjunto de hiperparámetros sin dependencia entre hiperparámetros, es decir, seleccionar los mejores que optimicen un modelo por sí solos.

El modelo que vamos a utilizar como base se trata de un modelo de red neuronal por capas donde las primeras tres capas son capas de Red Neuronal Convolucional para extraer las características más relevantes de la imagen. Hemos optado por tres capas ya que con más capas se podrían obtener características con mayor relevancia y mejor definidas pero el coste computacional también sería bastante mayor, y con menos capas el modelo pordía no obtener características suficientes para clasificar las muestras. Para estas capas convolucionales como hiperparámetro tenemos el número de neuronas que se utilizan en cada capa y el tamaño de la ventana de convolución que utiliza la capa. El modelo incluye después de cada capa convolucional una capa de MaxPooling para reducir la escala de parámetros de las capas manteniendo las características extraidas. Estas capas como hiperparámetro tienen el tamaño de ventana de MaxPooling. Finalmente, como últimas capas, el modelo incluye dos capas de Red Neuronal Densa que son las capas encargadas de aprender las características extraidas y clasificar las muestras según los distintos tipos de frutas. Hemos elegido utilizar dos capas por el mismo motivo explicado anteriormente, eligiendo una proporción adecuada entre coste computacional y capacidad de apredizaje del modelo. Estas capas tiene como hiperparámetro el número de neuronas que implementan aunque solo en la primera de ellas ya que en la última capa al tratarse de la capa de salida tiene qu tener tantas neuronas como posibles clases a identificar. Inicialmente hemos probado distintos modelos con más y menos capas y con valores de los hiperparámetros más extremos donde se obtenían o muy malos resultados o el tiempo de ejecución era excesivo. Finalmente hemos acotado los posibles hiperparámetros al conjunto que mostramos donde las diferencias tampoco son tan grandes en cuanto al rendimiento del modelo.

Vamos a necesitar uan función que dados los hiperparámetros nos cree el modelo que vamosa  entrenar y evaluar.

In [ ]:
def create_model(conv_neruons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape):
    local_model = Sequential()

    # Capa convolucional 1
    local_model.add(Conv2D(conv_neurons[0], conv_window[0], activation='relu', input_shape=input_shape))
    local_model.add(MaxPooling2D(num_maxPooling[0]))

    # Capa convolucional 2
    local_model.add(Conv2D(conv_neurons[1], conv_window[1], activation='relu'))
    local_model.add(MaxPooling2D(num_maxPooling[1]))

    # Capa convolucional 3
    local_model.add(Conv2D(conv_neurons[2], conv_window[2], activation='relu'))
    local_model.add(MaxPooling2D(num_maxPooling[2]))

    # Capa de aplanamiento
    local_model.add(Flatten())

    # Capa completamente conectada
    local_model.add(Dense(dense_neurons, activation='relu'))

    # Capa de salida
    local_model.add(Dense(num_classes, activation='softmax'))

    return local_model

Definimos la función para utilizar grid_search

In [ ]:
def single_grid_search(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape, X, y, epochs, batch_size):
    # Creamos el modelo con los hiperparámetros introducidos
    local_model = create_model(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape)

    local_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=[
            tf.keras.metrics.CategoricalAccuracy(),
            tf.keras.metrics.F1Score(),
            CustomSensitivity(),
            CustomScoreMetric()
        ],
    )

    # Define Early Stopping
    early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

    # Obtener conjuntos de validación y train
    trainX, valX, trainY, valY = train_test_split(X, y, test_size=0.40)

    # Entrenar el modelo
    history = local_model.fit(
      trainX, trainY,
      epochs=epochs,
      batch_size=batch_size,
      verbose = 0, # Cantidad de información a mostrar mientras se entrena
      callbacks=[early_stopping]
    )

    # Evaluar el modelo
    return local_model.evaluate(valX,valY)

Valores de entrada y salida de los modelos:

In [ ]:
input_shape = trainX[0].shape
num_classes = len(map_labels)

Valores de los hiperparámetros a probar:

In [ ]:
# Valores de los hiperparámetros para el modelo base
conv_neurons = [16,32,64]
conv_window = [[3,3],[3,3],[3,3]]
num_maxPooling = [[2,2],[2,2],[2,2]]
dense_neurons = 64
batch_size = 96

# Valores de los  distintos hiperparámetros
conv_neurons_search = [[16,32,64],[32,64,128]]
conv_window_search = [[[2,2],[2,2],[2,2]],[[3,3],[3,3],[3,3]],[[5,5],[5,5],[5,5]]]
num_maxPooling_search = [[[2,2],[2,2],[2,2]],[[3,3],[3,3],[3,3]]]
dense_neurons_search = [16,32,64,128]
batch_size_search = [32,64,96,128]
epochs = 2

##Selección de los hiperparámetros:

> Hiperparámetro del tamaño de ventana de convolución

In [ ]:
funcionPerdidaTabla = []
accuracyTabla = []
f1Tabla = []
sensitivityTabla = []
customScoreTabla = []

for conv_window in conv_window_search:
    # Entrenar y evaluar
    results = single_grid_search(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape, trainX, trainY_encoded, epochs, batch_size)

    accuracyTabla.append(results[1])
    f1Tabla.append(results[2].mean())
    funcionPerdidaTabla.append(results[0])
    sensitivityTabla.append(results[3])
    customScoreTabla.append(results[4].mean())


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Ventana Convolucion': conv_window_search,'Error de validación (función de pérdida)':funcionPerdidaTabla,
                           'Accuracy':accuracyTabla,'F1-Score':f1Tabla,'Sensitivity':sensitivityTabla, 'Métrica personalizada':customScoreTabla})
display(tableFrame)

127/127 [==============================] - 17s 129ms/step - loss: 0.2155 - categorical_accuracy: 0.9315 - f1_score: 0.9307 - sensitivity: 0.9234 - custom_score: 0.9302


,Ventana Convolucion,Error de validación (función de pérdida),Accuracy,F1-Score,Sensitivity,Métrica personalizada
0,"[[2, 2], [2, 2], [2, 2]]",0.061926,0.99110,0.991113,0.986650,0.990435
1,"[[3, 3], [3, 3], [3, 3]]",0.047443,0.98937,0.988957,0.985909,0.988789
2,"[[5, 5], [5, 5], [5, 5]]",0.215472,0.93152,0.930654,0.923362,0.930167


A partir de los resultados obtenidos, podemos ver que los dos primeros valores obtienen unos resultados en las métricas mejores que el tercer valor, pero el segundo valor obtiene una función de pérdida menor, lo que indica que el modelo se ha ajustado mejor a la función de pérdida con ese valor. Por lo tanto elegimos ese hiperparámetro par el modelo.

> Hiperparámetro del Número de Neuronas en las capas de Convolución

In [ ]:
funcionPerdidaTabla = []
accuracyTabla = []
f1Tabla = []
sensitivityTabla = []
customScoreTabla = []

for conv_neurons in conv_neurons_search:
    # Entrenar y evaluar
    results = single_grid_search(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape, trainX, trainY_encoded, epochs, batch_size)

    accuracyTabla.append(results[1])
    f1Tabla.append(results[2].mean())
    funcionPerdidaTabla.append(results[0])
    sensitivityTabla.append(results[3])
    customScoreTabla.append(results[4].mean())


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Neuronas Convolucionales': conv_neurons_search,'Error de validación (función de pérdida)':funcionPerdidaTabla,
                           'Accuracy':accuracyTabla,'F1-Score':f1Tabla,'Sensitivity':sensitivityTabla, 'Métrica personalizada':customScoreTabla})
display(tableFrame)

127/127 [==============================] - 32s 251ms/step - loss: 0.0457 - categorical_accuracy: 0.9894 - f1_score: 0.9887 - sensitivity: 0.9874 - custom_score: 0.9890


,Neuronas Convolucionales,Error de validación (función de pérdida),Accuracy,F1-Score,Sensitivity,Métrica personalizada
0,"[16, 32, 64]",0.041732,0.988381,0.989220,0.986156,0.988173
1,"[32, 64, 128]",0.045693,0.989370,0.988686,0.987392,0.988970


A partir de los resultados obtenidos, podemos ver que el segundo hiperparámetro probado obtiene ligeramente mejores resultados en las métricas, como era de esperar ya que con un número mayor de neuronas el modelo es capaz de aprender más y obtener mejores resultados. Como una de las partes más importantes del modelo es extraer los rasgos característicos de cada clase, vamos a elegir el segundo valor para que el modelo tenga más capacidad a la hora de identificar las frutas del conjunto incial y las que generemos nosotros.

> Hiperparámetro del Tamaño de Ventana de MaxPooling

In [ ]:
funcionPerdidaTabla = []
accuracyTabla = []
f1Tabla = []
sensitivityTabla = []
customScoreTabla = []

for num_maxPooling in num_maxPooling_search:
    # Entrenar y evaluar
    results = single_grid_search(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape, trainX, trainY_encoded, epochs, batch_size)

    accuracyTabla.append(results[1])
    f1Tabla.append(results[2].mean())
    funcionPerdidaTabla.append(results[0])
    sensitivityTabla.append(results[3])
    customScoreTabla.append(results[4].mean())


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Ventana MaxPooling': num_maxPooling_search,'Error de validación (función de pérdida)':funcionPerdidaTabla,
                           'Accuracy':accuracyTabla,'F1-Score':f1Tabla,'Sensitivity':sensitivityTabla, 'Métrica personalizada':customScoreTabla})
display(tableFrame)

127/127 [==============================] - 16s 121ms/step - loss: 0.1244 - categorical_accuracy: 0.9614 - f1_score: 0.9619 - sensitivity: 0.9540 - custom_score: 0.9604


,Ventana MaxPooling,Error de validación (función de pérdida),Accuracy,F1-Score,Sensitivity,Métrica personalizada
0,"[[2, 2], [2, 2], [2, 2]]",0.044838,0.985414,0.984493,0.984425,0.985128
1,"[[3, 3], [3, 3], [3, 3]]",0.124393,0.961434,0.961853,0.954017,0.960384


A partir de los resultados obtenidos, podemos ver que el primer hiperparámetro probado obtiene mejores resultados en las métricas, como era de esperar ya que con una reducción de dimensión de menor escala, el modelo mantiene más características extraidas entre capa y capa. Por lo tanto elegimos ese hiperparámetro par el modelo.

> Hiperparámetro del Número de Neuronas

In [ ]:
funcionPerdidaTabla = []
accuracyTabla = []
f1Tabla = []
sensitivityTabla = []
customScoreTabla = []

for dense_neurons in dense_neurons_search:
    # Entrenar y evaluar
    results = single_grid_search(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape, trainX, trainY_encoded, epochs, batch_size)

    accuracyTabla.append(results[1])
    f1Tabla.append(results[2].mean())
    funcionPerdidaTabla.append(results[0])
    sensitivityTabla.append(results[3])
    customScoreTabla.append(results[4].mean())


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Número neuronas': dense_neurons_search,'Error de validación (función de pérdida)':funcionPerdidaTabla,
                           'Accuracy':accuracyTabla,'F1-Score':f1Tabla,'Sensitivity':sensitivityTabla, 'Métrica personalizada':customScoreTabla})
display(tableFrame)

127/127 [==============================] - 16s 126ms/step - loss: 0.1447 - categorical_accuracy: 0.9533 - f1_score: 0.9512 - sensitivity: 0.9392 - custom_score: 0.9509


,Número neuronas,Error de validación (función de pérdida),Accuracy,F1-Score,Sensitivity,Métrica personalizada
0,16,3.357697,0.087268,0.019572,0.022250,0.067361
1,32,1.152449,0.665019,0.614018,0.563906,0.642202
2,64,0.267995,0.912732,0.904117,0.904079,0.910142
3,128,0.144684,0.953276,0.951239,0.939184,0.950856


A partir de los resultados obtenidos, podemos ver que el tercer y cuarto hiperparámetros probados obtienen mejores resultados en las métricas, como era de esperar ya que cuantas más neuronas se usen en la red neuronal, mayor capacidad de aprendizaje tiene el modelo. Para evitar un sobreajuste y disminuir el coste computacional del entrenamiento, vamos a elegir el tercer hiperparámetro que tiene la mitad de neuronas que el cuarto, ya que obtienen un resultado muy parecido que ya es bastante bueno.

> Hiperparámetro del Tamaño de Batch

In [ ]:
funcionPerdidaTabla = []
accuracyTabla = []
f1Tabla = []
sensitivityTabla = []
customScoreTabla = []

for batch_size in batch_size_search:
    # Entrenar y evaluar
    results = single_grid_search(conv_neurons, conv_window, num_maxPooling, dense_neurons, num_classes, input_shape, trainX, trainY_encoded, epochs, batch_size)

    accuracyTabla.append(results[1])
    f1Tabla.append(results[2].mean())
    funcionPerdidaTabla.append(results[0])
    sensitivityTabla.append(results[3])
    customScoreTabla.append(results[4].mean())


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tamaño de Batch': batch_size_search,'Error de validación (función de pérdida)':funcionPerdidaTabla,
                           'Accuracy':accuracyTabla,'F1-Score':f1Tabla,'Sensitivity':sensitivityTabla, 'Métrica personalizada':customScoreTabla})
display(tableFrame)

127/127 [==============================] - 18s 140ms/step - loss: 0.0472 - categorical_accuracy: 0.9896 - f1_score: 0.9889 - sensitivity: 0.9879 - custom_score: 0.9893


,Tamaño de Batch,Error de validación (función de pérdida),Accuracy,F1-Score,Sensitivity,Métrica personalizada
0,32,0.114796,0.965389,0.961988,0.961187,0.964249
1,64,0.067747,0.976514,0.976653,0.973300,0.976053
2,96,0.035903,0.989370,0.989414,0.988133,0.989191
3,128,0.047196,0.989617,0.988942,0.987886,0.989256


A partir de los resultados obtenidos, podemos ver que el tercer y cuarto hiperparámetros probados obtienen ligeramente mejores resultados en las métricas, por lo tanto con estos tamaños de batch el modelo se ajusta mejor a los datos. Aún así, los resultados nos han mostrado que con tamaño 96 de batch el modelo obtiene una función de pérdida menor, es decir que el modelo ha sido capaz de ajustar sus predicciones mejor a la función de pérdida y como en el resto de métricas obtienen resultados muy parecidos vamso a tomar el tercer valor para este hiperparámetro.

De este modo hemos podido elegir los valores para los posibles hiperparámetros de forma experimental evaluando qué valores obtenian un mejor resultado para el modelo con los datos de entrenamiento.